In [95]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

In [96]:
data = pd.read_csv("data/one_hot_encoded_data.csv")
# Remove NaN values
data = data.dropna()

# Split the data into X and y
X = data.drop("Label", axis=1)
y = data["Label"]

In [97]:
# Split the data into training and testing sets
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)


In [98]:
numeric_features = ['Q1', 'Q2', 'Q4'] 


numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features)
    ],
    remainder='passthrough'  # This will keep all other columns unchanged
)

# Append classifier to preprocessing pipeline.
# Now we have a full prediction pipeline.
model = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression())])

In [99]:
# Drop categorical columns already encoded
categorical_columns_to_drop = ['Q3', 'Q5', 'Q6', 'Q7', 'Q8']
X_train = X_train.drop(columns=categorical_columns_to_drop, errors='ignore')
X_test = X_test.drop(columns=categorical_columns_to_drop, errors='ignore')

# Check for non-numeric columns
for column in X_train.columns:
    if X_train[column].dtype == 'object':
        print(f"Column {column} contains non-numeric values: {X_train[column].unique()[:5]}")

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

# Create a pipeline with just scaling and the model
model = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000))
])

# Fit the pipeline
model.fit(X_train, y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('classifier',
                 LogisticRegression(max_iter=1000, multi_class='multinomial'))])

In [101]:
test_accuracy = model.score(X_test, y_test)
test_accuracy

# # # Evaluate on validation data
val_accuracy = model.score(X_val, y_val)
val_accuracy 
# print(f"Best model validation accuracy: {best_val_accuracy:.4f}")

# # Only evaluate on test set once you've finalized your model
# final_accuracy = best_model.score(X_test, y_test)
# print(f"Final test accuracy: {final_accuracy:.4f}")

ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- Q5


In [103]:
from sklearn.model_selection import train_test_split

# First split: separate out the test set (20% of data)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Second split: divide remaining data into training (80%) and validation (20%)
# This gives us 64% training, 16% validation, 20% test of the original data
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.2, random_state=42)

# Now drop categorical columns that are already encoded
categorical_columns_to_drop = ['Q3', 'Q5', 'Q6', 'Q7', 'Q8']
X_train = X_train.drop(columns=categorical_columns_to_drop, errors='ignore')
X_val = X_val.drop(columns=categorical_columns_to_drop, errors='ignore')
X_test = X_test.drop(columns=categorical_columns_to_drop, errors='ignore')

# Create and fit the pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

model = Pipeline([
    ('scaler', StandardScaler()),
    ('classifier', LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=2000))
])

# Train on training data
model.fit(X_train, y_train)

# Evaluate on validation data
val_accuracy = model.score(X_val, y_val)
print(f"Validation accuracy: {val_accuracy:.4f}")

# Use validation set to tune hyperparameters
from sklearn.model_selection import GridSearchCV

param_grid = {
    'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
    'classifier__class_weight': [None, 'balanced']
}

grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

print(f"Best parameters: {grid_search.best_params_}")
print(f"Best validation score: {grid_search.best_score_:.4f}")

# Train best model
best_model = grid_search.best_estimator_
best_model.fit(X_train, y_train)

# Evaluate on validation data
best_val_accuracy = best_model.score(X_val, y_val)
print(f"Best model validation accuracy: {best_val_accuracy:.4f}")

# Only evaluate on test set once you've finalized your model
final_accuracy = best_model.score(X_test, y_test)
print(f"Final test accuracy: {final_accuracy:.4f}")

Validation accuracy: 0.8504
Best parameters: {'classifier__C': 0.01, 'classifier__class_weight': 'balanced'}
Best validation score: 0.8241
Best model validation accuracy: 0.8632
Final test accuracy: 0.8048
